In [1]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
import time
import traceback
from datetime import datetime

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.options.mode.chained_assignment = None

In [2]:
input_directory = 'D:\Program Files\Documents\CMU\BAR_EMA\output_data\prelim_data_0.csv'
submission_info_directory = "D:\Program Files\Documents\CMU\BAR_EMA\input_data\EMA Weekends_FINAL.xlsx"

In [3]:
df = pd.read_csv(input_directory)

C:\Users\Saurav\AppData\Local\Temp\ipykernel_16216\3544748107.py:1: DtypeWarning: Columns (12,13,14,18,59,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(input_directory)


### TO DO
- ~~Normalize Date Format~~
- ~~Float to Int~~
- ~~Create and Insert Columns (Need more input. Which date col. What dates?): 
    - ~~'Brst': 'Burst',
    - ~~'Wknd': 'Weekend',
    - ~~'Day': 'Day Within Weekend',
- MR_DrnkDur column
- MR_wake column
- ~~Consider additional columns to add:
    - ~~Date of Drinking Session
    - ~~Group
    

### Uniform Date Format

In [4]:
date_columns = [col for col in df.columns if 'date' in col.lower()]
formats = ['%d/%m/%Y', '%Y-%d-%m %H:%M:%S']
# Function to try parsing the date with multiple formats
def parse_date(date_str):
    for fmt in formats:
        try:
            return pd.to_datetime(date_str, format=fmt)
        except ValueError:
            # print(f'String: {date_str} Format failure: {fmt}')
            pass
    print(date_str)
    return pd.NaT  # Return NaT (Not-a-Time) for unparseable dates

# df[(df['TrigDate'].str.contains('-'))]['TrigDate'].sort_values(ascending=False)
# df['TrigDate'].sample(5)

# Apply the parse_date function to the 'DateColumn'
for date_col in date_columns:
    df[date_col] = df[date_col].apply(parse_date)

### Remove Decimal Points

In [5]:
int_columns = ['SubID','MR_FlSlp', 'MR_SlpQlt', 'MR_DrnkNum',
       'MR_DrnkDur', 'MR_rlx', 'MR_hngvr', 'MR_Embrssd', 'MR_agrsv', 'MR_vom',
       'MR_hrt', 'MR_rmbr', 'MR_work', 'MR_rude', 'MR_soc', 'MR_exprs',
       'MR_guilt', 'MR_expctd', 'MR_mood', 'MR_mrng', 'MR_prmsd', 'MR_more',
       'MR_fght', 'MR_enrgtc', 'MR_PsdOut', 'MR_MsdClss', 'MR_Impls',
       'MR_sleep', 'MR_weed', 'MR_plan', 'chr', 'anoy', 'upbt', 'sad', 'irit',
       'hap', 'bor', 'cnt', 'RP_enrgz', 'RP_excit', 'RP_sedat', 'RP_slotht',
       'RP_slug', 'RP_up', 'RP_crntloc', 'RP_wthothr',
       'RP_NumOth', 'RP_OthTyp', 'RP_EnjInt', 'RP_ActCld',
       'RP_ActAcc', 'RP_OthCld', 'RP_OthAcc', 'RP_Vrtl', 'RP_AlnLke',
       'RP_AlnGd', 'RP_WshOth', 'RP_lnly', 'RP_rjct', 'RP_incl', 'RP_acpt',
       'RP_vrtint', 'RP_vrtoth', 'RP_vrtwho', 'RP_vrtenj', 'RP_vrtcld',
       'RP_VrtAcc', 'RP_VrtOthCld', 'RP_VrtOthAcc', 'RP_NotInt', 'RP_WshInt',
       'RP_lonely', 'RP_reject', 'RP_included', 'RP_accepted', 'RP_said',
       'RP_spent', 'RP_impt', 'RP_spur', 'RP_Drnk']

for col in int_columns:
    df[col] = df[col].astype('Int64', errors='ignore')

### Convert Date to UTC

In [6]:
df['SubDate'] = df['SubDate'].astype('datetime64[ns]')

### New Columns: Burst,Weekend, Day Within Weekend

In [7]:
date_info_df = pd.read_excel(submission_info_directory, engine='openpyxl')
date_info_df = date_info_df[~date_info_df['Sub ID'].isna()].reset_index(drop=True)
date_info_df['SubID'] = date_info_df['Sub ID'].astype('Int64', errors='ignore')
date_info_df = date_info_df.drop(columns=['Date of Drinking Session ', 'Group', 'Sub ID']).reset_index(drop=True)
date_info_df.sample()

,EMA Weekend 1,EMA Weekend 2,EMA Weekend 3,EMA Weekend 4,EMA Weekend 5,EMA Weekend 6,SubID
169,Thurs 5/16/2019-Sun 5/19/2019,Thurs 5/23/2019-Sun 5/26/2019,Thurs 11/14/19 - Sun 11/17/19,Thurs 11/21/19 - Sun 11/24/19,Thurs 5/14/20 - Sun 5/17/20,Thurs 5/21/20 - Sun 5/24/20,2349


In [8]:
date_info_df = pd.melt(date_info_df, id_vars=['SubID'], var_name='Weekend', value_name='Submission_Range')
date_info_df['Weekend'] = date_info_df['Weekend'].str.replace('EMA Weekend', '').astype(int)

def expand_year(d):
    date_parts = d.split('/')
    if len(date_parts)==3:
        year = date_parts[2]
        if len(year)==2:
            year = f'20{year}'
            d = f'{d[:-2]}{year}'
    return d
    
def expand_date_range(row):
    
    try:
        if isinstance(row['Submission_Range'], datetime):
            return []
        date_parts = row['Submission_Range'].split('-')
        if len(date_parts)<=1:
            return []
        start_date_parts = date_parts[0].strip().split(' ')
        end_date_parts = date_parts[1].strip().split(' ')
        
        start_date = start_date_parts[1]
        end_date = end_date_parts[1] if len(end_date_parts)==2 else end_date_parts[0]
        
        start_date = expand_year(start_date)
        end_date = expand_year(end_date)
            
        if len(start_date.split('/'))==2 and len(end_date.split('/'))==2:
            print(f'SubID: {row["SubID"]} {row["Submission_Range"]} Range End Date Year: {end_date} Start Year: {start_date}')
            return []
        elif len(start_date.split('/'))==2:
            end_date_month, end_date_date ,end_date_year = end_date.split('/')
            start_date_year = end_date_year 
            if end_date_month=='1' and end_date_date in['1','2','3']:
                start_date_year = str(int(end_date_year) - 1)
            start_date = start_date + f'/{start_date_year}'
        
        elif len(end_date.split('/'))==2:
            start_date_month, start_date_date, start_date_year = start_date.split('/')
            end_date_year = start_date_year
            if start_date_month=='12' and start_date_date in ['29','30','31']:
                end_date_year = str(int(start_date_year)+1)
            
            end_date = end_date + f'/{end_date_year}'
        
        try:
            start_date = pd.to_datetime(start_date, format='%m/%d/%Y')
            end_date = pd.to_datetime(end_date, format='%m/%d/%Y')
        except ValueError:
            print(f'{start_date} and {end_date}')
            
        # date_range = pd.date_range(start_date, end_date, freq='D')
        date_range = pd.date_range(start_date, periods=4, freq='D')
        res = [date.strftime('%a %m/%d/%y') for date in date_range]
        
        if len(res)!=4:
            print(row)
            print(res)
            print(f"!! RANGE IS {row['Submission_Range']}, DATES ARE {start_date} and {end_date}")
        
        return res
    except Exception as e:
        traceback.print_exc()
        display(row)
        print(e)
        return []

date_info_df['Submission_Date'] = date_info_df.apply(expand_date_range, axis=1)
date_info_df = date_info_df.explode('Submission_Date').reset_index(drop=True)
date_info_df['Burst'] = np.ceil(date_info_df['Weekend'] / 2).astype(int)
date_info_df['Day'] = pd.to_datetime(date_info_df['Submission_Date'], format='%a %m/%d/%y').dt.strftime('%a')
date_info_df = date_info_df[~date_info_df['Submission_Date'].isna()]
date_info_df['Submission_Date'] = date_info_df['Submission_Date'].astype('datetime64[ns]')
# date_info_df = date_info_df.drop(columns=['Submission_Range', 'Submission_Date']).reset_index(drop=True)

C:\Users\Saurav\AppData\Local\Temp\ipykernel_16216\2367939162.py:75: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  date_info_df['Submission_Date'] = date_info_df['Submission_Date'].astype('datetime64[ns]')


In [9]:
date_info_df = date_info_df.drop(columns=['Submission_Range'])
res_df = df.merge(date_info_df, left_on=['SubID','SubDate'], right_on=['SubID', 'Submission_Date'])
res_df = res_df.drop(columns=['Submission_Date'])
res_df = res_df[['SubID', 'Burst', 'Weekend', 'Day',  'RespID', 'RespType', 'UserID', 'SurvName', 'SurvType',
       'InitDate', 'InitTime', 'SubDate', 'SubTime', 'TrigDate', 'TrigTime',
       'MR_wake', 'MR_rise', 'MR_bed', 'MR_FlSlp', 'MR_SlpQlt', 'MR_DrnkNum',
       'MR_DrnkDur', 'MR_rlx', 'MR_hngvr', 'MR_Embrssd', 'MR_agrsv', 'MR_vom',
       'MR_hrt', 'MR_rmbr', 'MR_work', 'MR_rude', 'MR_soc', 'MR_exprs',
       'MR_guilt', 'MR_expctd', 'MR_mood', 'MR_mrng', 'MR_prmsd', 'MR_more',
       'MR_fght', 'MR_enrgtc', 'MR_PsdOut', 'MR_MsdClss', 'MR_Impls',
       'MR_sleep', 'MR_weed', 'MR_plan', 'chr', 'anoy', 'upbt', 'sad', 'irit',
       'hap', 'bor', 'cnt', 'RP_enrgz', 'RP_excit', 'RP_sedat', 'RP_slotht',
       'RP_slug', 'RP_up', 'RP_crntloc', 'RP_othrloc', 'RP_wthothr',
       'RP_NumOth', 'RP_OthTyp', 'RP_OthWho', 'RP_EnjInt', 'RP_ActCld',
       'RP_ActAcc', 'RP_OthCld', 'RP_OthAcc', 'RP_Vrtl', 'RP_AlnLke',
       'RP_AlnGd', 'RP_WshOth', 'RP_lnly', 'RP_rjct', 'RP_incl', 'RP_acpt',
       'RP_vrtint', 'RP_vrtoth', 'RP_vrtwho', 'RP_vrtenj', 'RP_vrtcld',
       'RP_VrtAcc', 'RP_VrtOthCld', 'RP_VrtOthAcc', 'RP_NotInt', 'RP_WshInt',
       'RP_lonely', 'RP_reject', 'RP_included', 'RP_accepted', 'RP_said',
       'RP_spent', 'RP_impt', 'RP_spur', 'RP_Drnk']]

In [17]:
res_df = res_df.sort_values(by=['SubID','SubDate','SurvName']).reset_index(drop=True)

In [18]:
res_df.to_csv('D:\Program Files\Documents\CMU\BAR_EMA\output_data\data_draft.csv', index=False)

### MR_DrnkDur Column

### MR_wake

### See next jupyter part....